In [1]:
import nglview
import ipywidgets
import os
import zipfile

## Step 1: Protein and ligand selection

In [5]:
# DOI: 10.2210/pdb3HTB/pdb
# 2-propylphenol in complex with T4 lysozyme
pdbCode = "3HTB"
ligandCode = "JZ4"

mol_charge = 0

proteinFile = pdbCode+'.pdb'
ligandFile = ligandCode+'.pdb'
complexFile = pdbCode+'_'+ligandCode+'.pdb'

## Step 2: Downloading Protein

From: RCSB PDB database

In [5]:
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
pdb_output = pdbCode+'.orig.pdb'

prop = {
    'pdb_code': pdbCode,
    'filter': False
}

# Downloading 3HTB from the Protein Data Bank
pdb(output_pdb_path=pdb_output, properties=prop)

2021-11-02 21:36:14,766 [MainThread  ] [INFO ]  Downloading: 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent


/home/dhnchandan/anaconda3/envs/nb_md_analysis/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2021-11-02 21:36:17,580 [MainThread  ] [INFO ]  Writting pdb to: 3HTB.orig.pdb


0

## Step 3: Extracting Protein, Ligand and Protein-Ligand Complex

In [4]:
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb

prop = {
    'heteroatoms' : [{"name": ligandCode}]
}

extract_heteroatoms(
    input_structure_path=pdb_output,
    output_heteroatom_path=ligandFile,
    properties=prop
)

extract_molecule(
    input_structure_path=pdb_output,
    output_molecule_path=proteinFile
)

# Concat protein and igand into one file
cat_pdb(
    input_structure1=proteinFile,
    input_structure2=ligandFile,
    output_structure_path=complexFile
)

print(proteinFile, ligandFile, complexFile)

## Step 4: Visualizing 3D structures

In [7]:
view_protein = nglview.show_structure_file(proteinFile)
view_protein._remote_call('setSize', target='Widget', args=['350px','400px'])
view_protein.camera='orthographic'
view_protein

view_ligand = nglview.show_structure_file(ligandFile)
view_ligand.add_representation(repr_type='ball+stick')
view_ligand._remote_call('setSize', target='Widget', args=['350px','400px'])
view_ligand.camera='orthographic'
view_ligand

view_complex = nglview.show_structure_file(complexFile)
view_complex.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view_complex._remote_call('setSize', target='Widget', args=['350px','400px'])
view_complex.camera='orthographic'
view_complex

ipywidgets.HBox([view_protein, view_ligand, view_complex])

## Step 5: Fixing protein structure

- missing side-chain atoms
- missing backbone atoms, heteroatoms and modified residues

In [8]:
from biobb_model.model.fix_side_chain import fix_side_chain

fixed_pdb_out = pdbCode + '_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=proteinFile, output_pdb_path=fixed_pdb_out)

2021-11-02 22:06:01,503 [MainThread  ] [INFO ]  Not using any container
2021-11-02 22:06:01,503 [MainThread  ] [INFO ]  check_structure -i 3HTB.pdb -o 3HTB_fixed.pdb --force_save fixside --fix ALL

2021-11-02 22:06:01,900 [MainThread  ] [INFO ]  Exit code 0

2021-11-02 22:06:01,901 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure 3HTB.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  163
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1300

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Fina

0

## Step 6: Creating topology

- Force field: amber99sb-ildn
- Water molecule type: spc/e

In [10]:
from biobb_md.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode+'_pdb2gmx_top.zip'

prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce',
    'gmx_path': '/usr/local/gromacs/bin/gmx' # latest local gromacs installation
}

pdb2gmx(
    input_pdb_path=fixed_pdb_out,
    output_gro_path=output_pdb2gmx_gro,
    output_top_zip_path=output_pdb2gmx_top_zip,
    properties=prop
)

2021-11-02 22:13:29,172 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20212 version detected
2021-11-02 22:13:29,173 [MainThread  ] [INFO ]  Not using any container
2021-11-02 22:13:29,174 [MainThread  ] [INFO ]  /usr/local/gromacs/bin/gmx -nobackup -nocopyright pdb2gmx -f 3HTB_fixed.pdb -o 3HTB_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2021-11-02 22:13:30,780 [MainThread  ] [INFO ]  Exit code 0

2021-11-02 22:13:30,781 [MainThread  ] [INFO ]  Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b

going to rename amber99sb-ildn.ff/dna.r2b

going to rename amber99sb-ildn.ff/rna.r2b
Reading 3HTB_fixed.pdb...
Read '', 1364 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 163 residues with 1364 atoms

  chain  #res #atoms

  1 'A'   163   1364  

there were 0 atoms with zero occupancy and 178 atoms with          occu

0

## Step 7: Preparing ligand

- Force field: amberGAFF


### Step 7.a: Add hydrogen atom

In [6]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

output_reduce_h = ligandCode+'.reduce.H.pdb'

prop = {
    'nuclear' : 'true'
}

reduce_add_hydrogens(
    input_path=ligandFile,
    output_path=output_reduce_h,
    properties=prop
)

2021-11-03 06:04:29,096 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:04:29,097 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.pdb > JZ4.reduce.H.pdb

2021-11-03 06:04:29,236 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:04:29,237 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "/home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.pdb"
Database of HETATM connections: "/home/dhnchandan/anaconda3/envs/nb_md_analysis//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Si

0

### Step 7.b: Energetically minimize the system

In [7]:
from biobb_chemistry.babelm.babel_minimize import babel_minimize

output_babel_min = ligandCode+'.H.min.mol2'

prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}

babel_minimize(
    input_path=output_reduce_h,
    output_path=output_babel_min,
    properties=prop
)

2021-11-03 06:06:19,699 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2021-11-03 06:06:19,700 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2021-11-03 06:06:19,701 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2021-11-03 06:06:19,701 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2021-11-03 06:06:19,702 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2021-11-03 06:06:19,703 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2021-11-03 06:06:19,703 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:06:19,704 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.reduce.H.pdb -omol2 > JZ4.H.min.mol2

2021-11-03 06:06:20,287 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:06:20,288 [MainThread  ] [INFO ]  
A T O M   T

0

### Step 7.c: Visualizing 3D structures

In [8]:
# Original structure view
view_original = nglview.show_structure_file(ligandFile)
view_original.add_representation(repr_type='ball+stick')
view_original._remote_call('setSize', target='Widget', args=['350px','400px'])
view_original.camera='orthographic'
view_original

# After adding hydrogen view
view_h_added = nglview.show_structure_file(output_reduce_h)
view_h_added.add_representation(repr_type='ball+stick')
view_h_added._remote_call('setSize', target='Widget', args=['350px','400px'])
view_h_added.camera='orthographic'
view_h_added

# After energy minimization view
view_e_minimized = nglview.show_structure_file(output_babel_min)
view_e_minimized.add_representation(repr_type='ball+stick')
view_e_minimized._remote_call('setSize', target='Widget', args=['350px','400px'])
view_e_minimized.camera='orthographic'
view_e_minimized

ipywidgets.HBox([view_original, view_h_added, view_e_minimized])

### Step 7.d: Generating ligand topology

In [9]:
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

output_acpype_gro = ligandCode+'params.gro'
output_acpype_itp = ligandCode+'params.itp'
output_acpype_top = ligandCode+'params.top'
output_acpype = ligandCode+'params'

prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

acpype_params_gmx(
    input_path=output_babel_min, 
    output_path_gro=output_acpype_gro,
    output_path_itp=output_acpype_itp,
    output_path_top=output_acpype_top,
    properties=prop
)

2021-11-03 06:12:26,193 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:12:26,194 [MainThread  ] [INFO ]  acpype -i /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.H.min.mol2 -b JZ4params.U15Loq -n 0

2021-11-03 06:12:27,738 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:12:27,739 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2021 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'JZ4'....
Checking parameters for unit 'JZ4'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing NE

0